In [2]:
from yelpapi import YelpAPI
import json
import pandas as pd
from pandas.io.json import json_normalize
import numpy

api_key = 'Z41R0DQquIn-_Y88pmgfAoavZA_kOFv-96EtbOcnGGdPVwqik70FkOXfIy_CRlcikfZ7nuzDtbx18DYg_vKJVJs32T3x7Zgx2y1bSxNGumP2U2S6bYtPBAOig4G3W3Yx'
yelp_api = YelpAPI(api_key)


In [3]:
# Ballparked coordinates of 4 vertices of Zip Code 60637.
upper_left_coordinate = [41.800792, -87.628796]  # [latitude, longitude]
upper_right_coordinate = [41.800792, -87.574454]
bottom_left_coordinate = [41.764310, -87.628796]
bottom_right_coordinate = [41.764310, -87.574454]
VDistance = abs(upper_left_coordinate[0] - bottom_left_coordinate[0])
HDistance = abs(upper_left_coordinate[1] - upper_right_coordinate[1])

# GPS distance to meters
# Converted using website below
# http://boulter.com/gps/distance/?from=41.800792+-87.628796&to=41.800792+-87.574454&units=k
HDistance_m = 4520  # Distance from upperleft to upperright in meters.
number_of_circles = 9
# Yelp only takes integer values for radius
circle_radius = round(HDistance_m / number_of_circles)  
circle_radius


502

In [4]:
# sample search
search_result = yelp_api.search_query(
    latitude=upper_right_coordinate[0],
    longitude=upper_right_coordinate[1],
    radius=circle_radius,
    limit=50)
df = pd.DataFrame.from_dict(search_result['businesses'], orient='columns')
df.head()


,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,rating,review_count,transactions,url
0,promontory-point-chicago,"[{'alias': 'beaches', 'title': 'Beaches'}, {'a...","{'latitude': 41.7964623, 'longitude': -87.576189}",(312) 747-6620,502.463461,HQuWqZhqprjqw2oXVTcZ2A,https://s3-media1.fl.yelpcdn.com/bphoto/7c_8Pc...,False,"{'address1': '5491 S S Shore Dr', 'address2': ...",Promontory Point,+13127476620,4.5,100,[],https://www.yelp.com/biz/promontory-point-chic...


In [ ]:
# This code only goes across, not downwards.
"""
def MoveRight_map (upper_left_coordinate, upper_right_coordinate,
        number_of_circles, circle_radius):
    ''' 
        Start at the upper left corner of the grid. 
        Get the information within a certain radius. 
        Move to the left. 
        Repeat until the longitude point is reached
    '''
    distance_between_circles_h = abs (
        upper_right_coordinate[1] - upper_left_coordinate[1]) / number_of_circles
    #latitude = upper_left_coordinate[0] 
    longitude = upper_left_coordinate[1]
    
    df = pd.DataFrame()
    for i in range (number_of_circles): 
        search_result = yelp_api.search_query(latitude=latitude, 
                                        longitude=longitude,
                                        radius = circle_radius,
                                        limit=50)
        df = df.append (pd.DataFrame.from_dict(
            search_result['businesses'], orient='columns'))
        longitude += distance_between_circles_h
    return df
        
#df = MoveRight_map (upper_left_coordinate, upper_right_coordinate,
    number_of_circles=9, circle_radius=circle_radius)
df.drop_duplicates('id') # remove duplicates
"""

In [ ]:
# Code for going downwards
"""
def MoveDown (upper_left_coordinate, upper_right_coordinate, bottom_left_coordinate, 
    number_of_circles, circle_radius):
    df = pd.DataFrame()
    distance_between_circles_v = abs (
        upper_left_coordinate[0] - bottom_left_coordinate[0]) / number_of_circles
    latitude = upper_left_coordinate[0] 
    for i in range (number_of_circles):
        df = df.append (MoveRight_map (upper_left_coordinate, upper_right_coordinate, 
                            number_of_circles, circle_radius))
        latitude -= distance_between_circles_v
    return df
"""

In [5]:
def MoveMap(upper_left_coordinate, upper_right_coordinate, bottom_left_coordinate,
            number_of_circles, circle_radius):
    '''
        Start at the upper left corner of the grid.
        Get the information within a certain radius.
        Move to the left.
        Repeat until the upper right point is reached.
        Move back to to the longitude of upper left point.
        Move downwards.
        Repeat.
    '''
    distance_between_circles_h = abs(
        upper_right_coordinate[1] - upper_left_coordinate[1]) / number_of_circles
    distance_between_circles_v = abs(
        upper_left_coordinate[0] - bottom_left_coordinate[0]) / number_of_circles
    latitude = upper_left_coordinate[0]
    longitude = upper_left_coordinate[1]

    df = pd.DataFrame()
    for v_step in range(number_of_circles):
        for h_step in range(number_of_circles):
            search_result = yelp_api.search_query(
                latitude=latitude,
                longitude=longitude,
                radius=circle_radius,
                limit=50)
            normalize = pd.DataFrame.from_dict(
                json_normalize(search_result['businesses']), orient='columns')
            # df = df.append (pd.DataFrame.from_dict(dfadd, orient='columns'))
            df = df.append(normalize)
            longitude += distance_between_circles_h
        longitude = upper_left_coordinate[1]
        latitude -= distance_between_circles_v
    return df.drop_duplicates(['id']).reset_index().drop('index',axis=1)


In [6]:
data_in_60637 = MoveMap(upper_left_coordinate, upper_right_coordinate, bottom_left_coordinate,
              number_of_circles=9, circle_radius=circle_radius)


In [7]:
data_in_60637.head()

,alias,categories,coordinates.latitude,coordinates.longitude,display_phone,distance,id,image_url,is_closed,location.address1,...,location.display_address,location.state,location.zip_code,name,phone,price,rating,review_count,transactions,url
0,jj-fish-and-chicken-chicago-3,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",41.796920,-87.630130,(773) 268-1600,535.446583,53z65ak2aZ2zrQEGAvq7NQ,https://s3-media3.fl.yelpcdn.com/bphoto/Oi2mxI...,False,5401 S Wentworth Ave,...,"[5401 S Wentworth Ave, Chicago, IL 60609]",IL,60609,JJ Fish and Chicken,+17732681600,$$,4.5,9,[],https://www.yelp.com/biz/jj-fish-and-chicken-c...
1,checkers-chicago,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",41.794779,-87.630321,(773) 268-5145,663.447196,hNxONY8We4_YEQc_p2WcBg,https://s3-media2.fl.yelpcdn.com/bphoto/g-U5QE...,False,5451 S Wentworth Ave,...,"[5451 S Wentworth Ave, Chicago, IL 60609]",IL,60609,Checkers,+17732685145,$,2.5,28,[],https://www.yelp.com/biz/checkers-chicago?adju...
2,waynes-bar-b-que-and-cajun-chicago,"[{'alias': 'chinese', 'title': 'Chinese'}]",41.796920,-87.630130,(773) 536-2282,572.995547,bmFyjzhFHdm1JL2Lx1iwqA,https://s3-media1.fl.yelpcdn.com/bphoto/Ft4B3V...,False,5401 S Wentworth Ave,...,"[5401 S Wentworth Ave, Chicago, IL 60609]",IL,60609,Wayne's Bar-B-Que and Cajun,+17735362282,$,4.0,6,[],https://www.yelp.com/biz/waynes-bar-b-que-and-...
3,new-grand-chinese-kitchen-chicago,"[{'alias': 'chinese', 'title': 'Chinese'}, {'a...",41.796920,-87.630130,(773) 536-8282,534.058034,LPFuagNX9Petb8vxMN42Ww,https://s3-media4.fl.yelpcdn.com/bphoto/5aAgCq...,False,5401 S Wentworth Ave,...,"[5401 S Wentworth Ave, Chicago, IL 60609]",IL,60609,New Grand Chinese Kitchen,+17735368282,$,3.0,10,"[delivery, pickup]",https://www.yelp.com/biz/new-grand-chinese-kit...
4,pizza-hut-chicago-33,"[{'alias': 'pizza', 'title': 'Pizza'}]",41.796920,-87.630130,(773) 285-5056,534.058034,5SFHn1n2oQfRiyo9n456sg,,False,5401 S Wentworth Ave,...,"[5401 S Wentworth Ave, Chicago, IL 60609]",IL,60609,Pizza Hut,+17732855056,NaN,3.0,3,[delivery],https://www.yelp.com/biz/pizza-hut-chicago-33?...


In [8]:
def get_yelp_data_by_location(location, number_of_calls):
    '''
        Given a location (ex. ZipCode),
        use Yelp API to retrieve data.
        Repeat by number of calls.
        Returns a dataframe.
    '''
    df = pd.DataFrame()
    for call in range (number_of_calls):
        search_result = yelp_api.search_query(
                    location=location,
                    limit=50)
        normalize = pd.DataFrame.from_dict(
                    json_normalize(search_result['businesses']), orient='columns')
        df = df.append(normalize)
    return df

location_data = get_yelp_data_by_location (
                    location='60637',
                    number_of_calls=2)
location_data.drop_duplicates(['id'])
# Calling any amount of times in the same location produces same results

,alias,categories,coordinates.latitude,coordinates.longitude,display_phone,distance,id,image_url,is_closed,location.address1,...,location.display_address,location.state,location.zip_code,name,phone,price,rating,review_count,transactions,url
0,museum-of-science-and-industry-chicago-3,"[{'alias': 'museums', 'title': 'Museums'}]",41.790577,-87.583069,(773) 684-1414,1620.209142,X7inrhYbhtjgJ4MrwtPlJQ,https://s3-media3.fl.yelpcdn.com/bphoto/EuFRTd...,False,5700 S Lake Shore Dr,...,"[5700 S Lake Shore Dr, Chicago, IL 60637]",IL,60637,Museum of Science and Industry,+17736841414,NaN,4.5,1021,[],https://www.yelp.com/biz/museum-of-science-and...
1,medici-on-57th-chicago,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {...",41.791267,-87.593766,(773) 667-7394,1039.364699,Zx87qmviYtwDOyBJah213g,https://s3-media2.fl.yelpcdn.com/bphoto/lzEU2Z...,False,1327 E 57th St,...,"[1327 E 57th St, Chicago, IL 60637]",IL,60637,Medici On 57th,+17736677394,$$,3.5,747,[],https://www.yelp.com/biz/medici-on-57th-chicag...
2,oriental-institute-museum-chicago-2,"[{'alias': 'museums', 'title': 'Museums'}]",41.789218,-87.597814,(773) 702-9520,702.046671,NfnmLKxavg9pFBgUxgAkJA,https://s3-media2.fl.yelpcdn.com/bphoto/1gZNys...,False,1155 E 58th St,...,"[1155 E 58th St, Chicago, IL 60637]",IL,60637,Oriental Institute Museum,+17737029520,NaN,4.5,77,[],https://www.yelp.com/biz/oriental-institute-mu...
3,valois-chicago,"[{'alias': 'breakfast_brunch', 'title': 'Break...",41.799602,-87.588372,(773) 667-0647,2075.136221,BwkeC6JB_4fIVpth2VUyQA,https://s3-media4.fl.yelpcdn.com/bphoto/ixqwiM...,False,1518 E 53rd St,...,"[1518 E 53rd St, Chicago, IL 60615]",IL,60615,Valois,+17736670647,$,4.0,512,[],https://www.yelp.com/biz/valois-chicago?adjust...
4,robust-coffee-lounge-chicago,"[{'alias': 'breakfast_brunch', 'title': 'Break...",41.780501,-87.596232,(773) 891-4240,414.327526,-Tp77Ce5OCpP1-KXUYtX9A,https://s3-media1.fl.yelpcdn.com/bphoto/ayXC_8...,False,6300 S Woodlawn,...,"[6300 S Woodlawn, Chicago, IL 60637]",IL,60637,Robust Coffee Lounge,+17738914240,$,4.0,210,"[pickup, delivery]",https://www.yelp.com/biz/robust-coffee-lounge-...
5,jackson-bark-chicago-2,"[{'alias': 'dog_parks', 'title': 'Dog Parks'}]",41.786308,-87.578461,(773) 272-1307,1807.707402,5MBgA082jDFf5DbrLN4-nQ,https://s3-media3.fl.yelpcdn.com/bphoto/W7LbxN...,False,6000 S Lake Shore Dr,...,"[6000 S Lake Shore Dr, Chicago, IL 60637]",IL,60637,Jackson Bark,+17732721307,NaN,4.5,135,[],https://www.yelp.com/biz/jackson-bark-chicago-...
6,daleys-restaurant-chicago,"[{'alias': 'diners', 'title': 'Diners'}, {'ali...",41.780285,-87.605310,(773) 643-6670,568.475314,_e40bDkQebc7bC44GJ2EsQ,https://s3-media1.fl.yelpcdn.com/bphoto/4H2LQY...,False,809 E 63rd St,...,"[809 E 63rd St, Chicago, IL 60637]",IL,60637,Daley's Restaurant,+17736436670,$$,4.0,146,[],https://www.yelp.com/biz/daleys-restaurant-chi...
7,plein-air-cafe-and-eatery-chicago-2,"[{'alias': 'cafes', 'title': 'Cafes'}]",41.790307,-87.596285,(773) 966-7531,844.691348,hB7yORLU0e6yEcpzA75cUw,https://s3-media1.fl.yelpcdn.com/bphoto/Ihx9Aq...,False,5751 S Woodlawn Ave,...,"[5751 S Woodlawn Ave, Chicago, IL 60637]",IL,60637,Plein Air Cafe & Eatery,+17739667531,$$,4.0,177,[],https://www.yelp.com/biz/plein-air-cafe-and-ea...
8,lstc-refectory-chicago-2,"[{'alias': 'cafeteria', 'title': 'Cafeteria'},...",41.795711,-87.598463,(773) 256-0702,1408.382358,6XTCsnjpa_fC_yjvvGbXQQ,https://s3-media4.fl.yelpcdn.com/bphoto/yNiE1p...,False,1100 E 55th St,...,"[1100 E 55th St, Chicago, IL 60615]",IL,60615,LSTC Refectory,+17732560702,$,5.0,52,[],https://www.yelp.com/biz/lstc-refectory-chicag...
9,frederick-c-robie-house-chicago,"[{'alias': 'landmarks', 'title': 'Landmarks & ...",41.789842,-87.595966,(312) 994-4000,816.744680,9uVC5BVplA1eVPk_koXDJg,https://s3-media1.fl.yelpcdn.com/bphoto/cQ8wlq...,False,5757 S Woodlawn Ave,...,"[5757 S Woodlawn Ave, Chicago, IL 60637]",IL,60637,Frederick C Robie House,+13129944000,NaN,4.0,80,[],https://www.yelp.com/biz/frederick-c-robie-hou...
